In [3]:
import pandas as pd
import numpy as np

In [4]:
#Tenemos la infomración en 2 idiommas así que creamos una lista de tuplas 
paises_europa = [("Albania", "Albania","AL"), ("Alemania", "Germany","DE"), ("Andorra", "Andorra"), ("Austria", "Austria","AT"), 
                ("Bélgica", "Belgium","BE"), ("Bielorrusia", "Belarus"), ("Bosnia y Herzegovina", "Bosnia and Herzegovina","BA"),
                ("Bulgaria", "Bulgaria","BG"), ("Croacia", "Croatia","HR"), ("Chipre", "Cyprus","CY"), ("Dinamarca", "Denmark","DK"), 
                ("Eslovaquia", "Slovakia","SK"), ("Eslovenia", "Slovenia","SI"), ("España", "Spain","ES"), ("Estonia", "Estonia","EE"), 
                ("Finlandia", "Finland","FI"), ("Francia", "France","FR"), ("Grecia", "Greece","EL"), ("Hungría", "Hungary","HU"), 
                ("Islandia", "Iceland","IS"), ("Irlanda", "Ireland","IE"), ("Italia", "Italy","IT"), ("Kosovo", "Kosovo"), 
                ("Letonia", "Latvia","LV"), ("Liechtenstein", "Liechtenstein"), ("Lituania", "Lithuania"), 
                ("Luxemburgo", "Luxembourg","LU"), ("Malta", "Malta","MT"), ("Moldavia", "Moldova"), ("Mónaco", "Monaco"), 
                ("Montenegro", "Montenegro","ME"), ("Noruega", "Norway"), ("Países Bajos", "Netherlands","NL"), ("Polonia", "Poland","PL"), 
                ("Portugal", "Portugal","PT"), ("Reino Unido", "United Kingdom","RU"), ("República Checa", "Czech Republic"), 
                ("República de Macedonia", "North Macedonia","MK"), ("Rumania", "Romania","RO"), ("Rusia", "Russia"), 
                ("San Marino", "San Marino"), ("Serbia", "Serbia","RS"), ("Suecia", "Sweden","SE"), ("Suiza", "Switzerland","CH"), 
                ("Ucrania", "Ukraine"), ("Vaticano", "Vatican City")]
#Creamos listas para filtrar las basses de datos por paises de Europa
pais =[x[0] for x in paises_europa]
country = [x[1] for x in paises_europa]


In [5]:
#Lectura archivo población mundial

world_pop = pd.read_csv('data/original/World_population.csv')
world_pop = world_pop.drop(columns=['Row','FIPS','GENC','Crude Birth Rate','Crude Death Rate','Net Migration Rate'])
total_world_pop22 = world_pop[world_pop['Year']==2022]['Population'].sum()
total_world_pop22

7905336896

In [6]:
world_waste = pd.read_csv('data/original/country_level_data_0.csv')
world_waste = world_waste.fillna(0)
world_waste.drop('gdp', inplace=True, axis=1)
world_waste

cols_numericas = world_waste.select_dtypes(include=[float,int]).columns

# calcula la suma de cada columna numérica en todas las filas
suma_por_fila = pd.DataFrame(world_waste[cols_numericas].sum(axis=0))

suma_por_fila[0].sum()

32242812195.21408

### Análisis y limpieza de datos sobre población, PIB y generación de basura de Europa
1. Fuente población [United States Census Bureau](www.census.gov)
2. Fuente PIB [Banco Mundial](https://datos.bancomundial.org/indicador/NY.GDP.PCAP.CD?end=2021&locations=EU&name_desc=false&start=2000)
3. Fuente [Eurostat](https://ec.europa.eu/eurostat/data/database)

In [7]:
# Datos obtenidos de census.gov United States Census Bureau sobre población
europe_population= pd.read_csv('data\original\Europe_population2000_2022.csv')
europe_population = europe_population.sort_values(ascending= False, by=['Population'])

#Eliminamos las columnas que no se necesitan de infomración
europe_population.drop(['Row','FIPS','GENC',],axis=1, inplace=True)
europe_population.drop(europe_population.iloc[:,4:], axis=1,inplace=True)

#Renombrar la columna
europe_population.rename(columns={'Country/Area Name':'Country Name'}, inplace=True)

#Eliminar columna de Area in Square
europe_population.drop('Area in Square Kilometers', axis=1, inplace=True)
europe_population.to_csv('data\clean\europe_population')

### Análisis y limpieza de datos relacionado con el PIB de Europa por país
Fuentes [Banco Mundial](https://datos.bancomundial.org/indicador/NY.GDP.PCAP.CD?end=2021&locations=EU&name_desc=false&start=2000)
& [International Monetary Fund](https://www.imf.org/external/datamapper/NGDPDPC@WEO/ADVEC/EUQ)

In [8]:
#Leemos el archivo extraído de IMF
pib_country = pd.read_excel('data\original\imf-dm-export-20230407.xls')

#Eliminamos todas las celdas que no contengan infomración o vacías
pib_country = pib_country.dropna() 

#Renombramos columna Country Name
pib_country.rename(columns={'GDP per capita, current prices (U.S. dollars per capita)':'Country Name'}, inplace=True)

#Filtramos los países de Europa de acuerdo a la lista de paises
pib_country = pib_country[pib_country['Country Name'].isin(country)]


#Eliminamos columnas de infomración de los años que no necesitamos
pib_country = pib_country.drop(pib_country.iloc[:, 1:21], axis=1)
pib_country = pib_country.drop(pib_country.iloc[:,24:], axis=1)

# Pasamos el valor de los años a filas por país con método melt
pib_country = pib_country.melt(id_vars=["Country Name"], var_name="Year", value_name="PIB_abs")

#Damos formato a los nombres de los países para que estén los nombres de todas las tablas en inglés
for i in range(len(pib_country)):
    pais_esp = pib_country.iloc[i]["Country Name"]
    for pair in paises_europa:
        if pais_esp == pair[0]:
            pib_country["Country Name"] = pib_country["Country Name"].replace([pair[0]],pair[1])

pib_country.to_csv('data\clean\pib_country')


### Análisis de datos sobre desperdicios sólidos en Europa y España
Fuente [Eurostat](https://ec.europa.eu/eurostat/data/database) & [Ecoembes](ecoembestransparencia.com/datos/envases-domesticos-ligeros/)

In [9]:
#Leemos archivo extraido de Eurostat sobre la basura
total_waste = pd.read_excel('data\original\env_wasmun__custom_5625727_spreadsheet.xlsx', sheet_name=3, skiprows=8)

#Eliminamos columna columnas que no son de utilidad
unwanted = total_waste.columns[total_waste.columns.str.startswith('Unnam')]
total_waste.drop(unwanted, axis=1, inplace=True)
total_waste.drop(['TIME'],axis=1, inplace=True)
total_waste.dropna(inplace=True)
total_waste.drop([1],axis=0,inplace=True)

#remplazamos los valores ':' por 0 para poder ser sumados 
total_waste.replace({":": 0}, inplace=True)
total_waste.replace({'Germany (until 1990 former territory of the FRG)':'Germany'}, inplace=True)

#Renombramos la columna del país
total_waste.rename(columns = {'TIME.1':'Country Name'},inplace=True)
total_waste.head()

c:\Users\Diana\anaconda3\envs\eda_env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2,Belgium,4831,4802,4981,4824,5059,5052,5115,5243,5141,...,4944,4867,4762,4643,4746,4672,4677.0,4779,8408,8795
3,Bulgaria,4998,4773,4715,4690,4619,4502,4387,4172,4486,...,3364,3135,3192,3011,2881,3080,2862.0,3080,2829,0
4,Czechia,3434,2798,2845,2857,2841,2954,3039,3025,3176,...,3233,3228,3261,3337,3580,5177,5248.0,5338,5814,5991
5,Denmark,3546,3519,3568,3618,3757,3990,4021,4313,4560,...,4508,4564,4558,4671,4757,4728,4715.0,4907,4744,4601
6,Germany,52810,52075,52772,49622,48434,46555,46426,47887,48367,...,49759,49570,51102,51625,52133,51790,50260.0,50612,53322,53748


In [10]:
#Estas serán las columnas que se mantendrán de pib_europe
cols_fijas = ['Country Name']
#se hace un list comprehension para los valores que necesitamos por cada año
cols_to_rows = [col for col in total_waste.columns if col.startswith('20')]

# Hacemos un melt para las columnas de los años y renombrar las columnas
total_waste = pd.melt(total_waste, id_vars=cols_fijas, value_vars=cols_to_rows, var_name='Year', value_name='Thousand_tonnes')

In [11]:
#Cambio de valores de Year
total_waste['Year'] = total_waste['Year'].astype('int')

In [12]:
total_waste.to_csv('data\clean\ total_waste')


### Análisis y limpieza de datos sobre basura en España por contenedores
Fuentes [INE](https://www.ine.es/jaxiT3/Tabla.htm?t=31304) & [Ecoembes](https://www.ecoembestransparencia.com/datos/envases-domesticos-ligeros/)

In [13]:
#Lectura de los datos
data = pd.read_excel('data/original/Recogida-2021.xlsx', sheet_name='BBDD Amarillo')
data2 = pd.read_excel('data/original/Recogida-2021.xlsx', sheet_name='BBDD Azul')
data3 = pd.read_excel('data/original/Seleccion-y-Reciclado-2021.xlsx', sheet_name='Tabla Seleccion')

,AÑO,Plásticos (Tn),Papel/Cartón (Tn),Metales (Tn),Cartón bebidas (Tn),Madera (Tn),Otros (Tn),Total general (Tn)
0,1999,22720.634000,162365.123266,53286.795000,5968.286000,2491.000000,NaN,2.468318e+05
1,2000,58144.467472,227350.763824,88908.098952,13377.150247,2984.000000,NaN,3.907645e+05
2,2001,85874.053460,330779.777242,128255.746550,22051.678340,5232.790000,NaN,5.721940e+05
3,2002,106022.591835,386020.755757,149411.799887,29592.284930,8534.382000,NaN,6.795818e+05
4,2003,140660.325310,432379.619210,183651.283000,35514.285420,9872.328980,NaN,8.020778e+05
5,2004,151239.720221,484743.225399,206066.572741,38967.824832,10403.807570,NaN,8.914212e+05
6,2005,185518.631070,541276.458247,221071.855609,45766.585048,10651.123390,NaN,1.004285e+06
7,2006,223251.039168,551560.021724,236032.005680,47116.709236,8382.754400,NaN,1.066342e+06
8,2007,252673.485710,607797.322234,236927.533494,57476.189800,5741.787000,NaN,1.160616e+06
9,2008,289823.284731,626841.369957,245950.496301,61469.896641,5550.613800,NaN,1.229636e+06


In [14]:
data['KG RECOGIDOS']= data['KG RECOGIDOS'].astype('int')
data['HABITANTES']= data['HABITANTES'].astype('int')
data2['Kg Recogidos'] = data2['Kg Recogidos'].astype('int')
data2['Nº Habitantes'] = data2['Nº Habitantes'].astype('int')
data2

,AÑO,CCAA,UNIDAD DE GESTIÓN,UG CODIGO,Kg Recogidos,Nº Habitantes
0,1998,ANDALUCIA,NaN,NaN,4228512,600847
1,1998,ASTURIAS,NaN,NaN,4852620,721222
2,1998,BALEARES,NaN,NaN,4129149,317212
3,1998,CASTILLA-LA MANCHA,NaN,NaN,568580,107089
4,1998,CATALUNA,NaN,NaN,4362720,4096080
...,...,...,...,...,...,...
15310,2021,REGION DE MURCIA,Ayuntamiento de Torre-Pacheco,UG1222,614730,36464
15311,2021,REGION DE MURCIA,Ayuntamiento de Totana,UG1212,637926,32529
15312,2021,REGION DE MURCIA,Ayuntamiento de Ulea,UG1762,6210,849
15313,2021,REGION DE MURCIA,Ayuntamiento de Villanueva del Rio Segura,UG1761,21100,3111


In [15]:
data= data.groupby(['AÑO','CCAA']).agg({'KG RECOGIDOS':'sum','HABITANTES':'sum'}).reset_index()
data2 =data2.groupby(['AÑO','CCAA']).agg({'Kg Recogidos':'sum','Nº Habitantes':'sum'}).reset_index()

In [16]:
merged = pd.merge(data, data2, on=['AÑO','CCAA'])
merged['Total KG Recogidos'] = merged['KG RECOGIDOS'] + merged['Kg Recogidos']
merged = merged.rename(columns={'KG RECOGIDOS':'Cont AMARILLO','Kg Recogidos':'Cont AZUL'})
merged
#merged.to_csv('data\clean\contenedores')

,AÑO,CCAA,Cont AMARILLO,HABITANTES,Cont AZUL,Nº Habitantes,Total KG Recogidos
0,1998,ANDALUCIA,2959304,238261,4228512,600847,7187816
1,1998,ASTURIAS,70260,250981,4852620,721222,4922880
2,1998,CATALUNA,6220,358403,4362720,4096080,4368940
3,1998,GALICIA,97260,109220,43410,7929,140670
4,1998,MADRID,13139520,1377467,4748282,3394223,17887802
...,...,...,...,...,...,...,...
397,2021,MADRID,190498589,6779883,126358355,6779885,316856944
398,2021,MELILLA,593480,87076,1301551,87076,1895031
399,2021,NAVARRA,16210215,607813,28051255,660298,44261470
400,2021,PAIS VASCO,53793766,2220504,69748761,2220504,123542527


In [17]:
#Selección de datos para plotear mapa
datos_mapa= merged

#Selección de columnas a utilizar
datos_mapa = datos_mapa.drop(['Cont AMARILLO','HABITANTES','Cont AZUL','Nº Habitantes'], axis=1)

#Filtrado de todas las CCAA por año 2021
datos_mapa = datos_mapa[datos_mapa['AÑO']==2021]

#Acomodo de nombres de comunidades en orden alfabético
datos_mapa.sort_values(by='CCAA', inplace=True)
datos_mapa = datos_mapa[['CCAA','Total KG Recogidos']]

#Guardar la infomración para plotera mapa en Qgis
datos_mapa.to_csv('data\clean\CCAA')

In [18]:
#Lectura de archivo original
population_per_ccaa = pd.read_csv('data/original/2853bsc.csv',index_col=0, sep=';', encoding='latin-1')

#Filtrado de Nans
population_per_ccaa.dropna(inplace=True)

#Reset index
population_per_ccaa =  population_per_ccaa.reset_index(drop='index')

#Eliminación de columna Sexo 
population_per_ccaa.drop('Sexo', inplace=True, axis=1)

#Renombrar la columna periodo por Año
population_per_ccaa.rename(columns={'Periodo':'Año'})

population_per_ccaa.rename(columns={'Comunidades y Ciudades Autï¿½nomas':'CCAA','Total':'Población'},inplace=True)

#transformación de columna población a int

population_per_ccaa['Población'] = population_per_ccaa['Población'].str.replace('.','').astype(int)

#guardar archivo ya limpio
population_per_ccaa.to_csv('data\clean\population_per_CCAA')


C:\Users\Diana\AppData\Local\Temp\ipykernel_37516\956414119.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  population_per_ccaa['Población'] = population_per_ccaa['Población'].str.replace('.','').astype(int)
